## ESPP 90S Final Project: Ammonia x Wind
### Alaisha Sharma and Rowen VonPlagenhoef

In [ ]:
import numpy as np

#### PARAMETERS

In [ ]:
# Lifetime of plant in yr
life_wind = 30
life_ammonia = 15

# WIND FARM

# Wind turbine size in kW
size_turbine = 2000
# Number of wind turbines
num_turbines = 20
# Wind farm capacity factor
capacity = 0.40
# Cost of wind power generation in $ / kW
cost_wind = 1400

# AMMONIA SYNTHESIS

# Daily production of ammonia
ammonia_output_daily = None
# Cost of input water in $ / ton H2O
cost_water = 1.064
# Energy usage of electrolyzers in kWh / ton NH3
energy_alkaline_electrolyzer = 62
energy_PEM_electrolyzer = 60
# Cost of nitrogen air separator in $-day / ton N2
cost_air_separator = 57119
# Cost of ammonia synthesizer in $-day / ton NH3
cost_synthesizer = 127327
# Cost of air compressor in $ 
cost_compressor = 1e6

# STORAGE

# Cost of ammonia storage in $ / ton
cost_storage_NH3 = 857
# Cost of hydrogen storage in $ / ton
cost_storage_H2 = None
# Cost of oxygen storage in $ / ton
cost_storage_O2 = None
# Cost of nitrogen storage in $ / ton
cost_storage_N2 = None

# CARBON EMISSIONS

# Carbon emissions of Haber Bosch process in tons CO2 / tons NH3
co2_haber_bosch = None
# Carbon emissions of fossil fuel electricity generation in tons CO2 / kW
co2_fossil_electricity = None
# Energy usage of traditional ammonia plant in kWh / ton NH3
energy_fossil_ammonia = None

# OTHER
interest_rate = 0.03
labor_fraction = 0.15

#### FUNCTIONS

In [ ]:
def calc_wind_power_output():
    # kWh / yr
    return size_turbine*num_turbines*capacity_factor*8760

In [ ]:
def calc_levelized_cost_wind():
    # yr
    n_periods = life_wind
    # kW * ($ / kW)
    cost_capital = size_turbine*num_turbines*cost_wind
    # $
    pmt_wind = np.pmt(interest_rate, life_wind, cost_capital)
    # kWh / yr
    wind_power_output = calc_wind_power_output()
    # $ / kWh
    return pmt_wind/wind_power_output

In [ ]:
def calc_cost_ammonia_synthesis(cost_electricity, 
                                ammonia_output_daily):
    H2_produced = (3/2)*ammonia_output_daily
    N2_produced = (1/2)*ammonia_output_daily
    # tons H2O * ($ / tons NH3)
    cost_H2O = ammonia_output_daily*cost_water
    # tons H2 * (kW / NH3 ton) * ($ / kW) 
    cost_H2 = H2_produced*energy_electrolyzer*cost_electricity
    # tons N2 * ($ / NH3 ton)
    cost_N2 = N2_produced*cost_air_separator
    # tons NH3 * (kW / NH3 ton) * ($ / kW )
    cost_NH3 = ammonia_output_daily*energy_compressor*cost_electricity
    # $ * (days / yr)
    return (cost_water+cost_H2+cost_N2+cost_NH3)*365

In [ ]:
def ammonia_storage_breakdown(ammonia_output_daily):
    storage_NH3 = 30*ammonia_output_daily
    storage_N2 = None
    storage_H2 = None
    storage_O2 = None
    return storage_H2, storage_O2, storage_N2, storage_NH3

In [2]:
def calc_cost_chemical_storage(storage_H2,
                               storage_O2,
                               storage_N2,
                               storage_NH3):
    # $ / ton NH3
    cost_H2 = storage_H2*cost_storage_H2
    # $ / ton NH3
    cost_O2 = storage_O2*cost_storage_O2
    # $ / ton NH3
    cost_N2 = storage_N2*cost_storage_N2
    # $ / ton NH3
    cost_NH3 = storage_NH3*cost_storage_NH3
    # $ / ton NH3
    return cost_H2+cost_O2+cost_N2+cost_NH3


In [ ]:
def calc_cost_ammonia_output(ammonia_output_daily):
    # $
    cost_synthesis = calc_cost_ammonia_synthesis(ammonia_output_daily)
    # tons
    H2, O2, N2, NH3 = ammonia_storage_breakdown(ammonia_output_daily)
    # $
    cost_storage = calc_cost_chemical_storage(H2, O2, N2, NH3)
    # $
    cost_labor = labor_fraction*(cost_ammonia*ammonia_output_daily+cost_storage)
    # $
    return cost_synthesis+cost_storage+cost_labor

In [ ]:
def calc_levelized_cost_ammonia():
    # yr
    n_periods = life_ammonia
    # $
    cost_capital = calc_cost_ammonia_output(ammonia_output_daily)*365
    # $
    pmt_ammonia = np.pmt(interest_rate, life_ammonia, cost_capital)
    # kWh / yr
    ammonia_output = ammonia_output_daily*365
    # $ / kWh
    return pmt_ammonia/ammonia_output

In [ ]:
def calc_co2_tax_ammonia(cost_co2, 
                         ammonia_produced):
    # (tons CO2 / kWh) * (kWh / ton NH3)
    co2_fossil_plant = co2_fossil_electricity*energy_fossil_ammonia
    # (tons CO2 / tons NH3) * tons NH3
    co2_emissions = (co2_haber_bosch+co2_fossil_plant)*ammonia_produced
    # (tons) * ($ / tons CO2)
    return co2_emissions*cost_co2